In [1]:
# Install the pycoco library as it is not part of the Kaggle environment
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-8mkxd4tv
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-8mkxd4tv
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=272558 sha256=e170624c7d934bffc4216295311f47675271070c17ee20a5eaf4c5a326257b52
  Stored in directory: /tmp/pip-ephem-wheel-cache-305zmjvq/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools


In [2]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import json
from pycocotools.coco import COCO

In [3]:
# Define the load images function
def loadImages(LOC, COLOUR=None):
    file_lst = os.listdir(LOC)
    
    # Read the images as gray or coloured
    image_dct = {}
    for file in file_lst:
        loc = LOC + file
        if COLOUR == 'gray':
            img = cv2.imread(loc, 0)
        else:
            img = cv2.imread(loc)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        
        
        # Save the images and their image ids into a dictionary
        image_dct[file[:6].strip('0')] = img
    
    # Read the json file into a coco indexer
    coco = COCO(LOC[:-7] + 'annotations.json')
        
    return image_dct, coco

In [4]:
# Load the image sets and their annotations
train_images, train_coco = loadImages('../input/food-recognition-challenge/val-v0.4/val/images/', 'gray')

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!


### TO DO:
https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-1-of-2-732712631047

Link to website on guide to read the images with their annotations using the coco indexer ('train_coco'). Implement a function to preview images with their annotations.